In [1]:
from crocoddyl import *
import pinocchio as pin
import numpy as np
from crocoddyl.diagnostic import displayTrajectory

In [2]:
# LOAD ROBOT
robot = loadHector()
robot.initViewer(loadModel=True)
robot.display(robot.q0)

robot.framesForwardKinematics(robot.q0)

rmodel = robot.model

In [12]:
def uavPlacementModel(targetPos, targetQuat, integrationStep, frameName):
    # ACTUATION MODEL
    distanceRotorCOG = 0.1525
    cf = 6.6e-5
    cm = 1e-6
    actModel = ActuationModelUAM(robot.model, distanceRotorCOG, cf, cm)

    # COST MODEL
    # Create a cost model per the running and terminal action model.
    runningCostModel = CostModelSum(rmodel, actModel.nu)
    terminalCostModel = CostModelSum(rmodel, actModel.nu)

    state = StatePinocchio(rmodel)
    SE3ref = pin.SE3()
    SE3ref.translation = targetPos.reshape(3,1)
    SE3ref.rotation = targetQuat.matrix()

    wBasePos  = [1]
    wBaseOri  = [1]
    wArmPos   = [1]
    wBaseVel  = [10]
    wBaseRate = [10]
    wArmVel   = [10] 
    
    stateWeights   = np.array(wBasePos * 3 + wBaseOri * 3 + wArmPos * (robot.model.nv - 6) + wBaseVel * robot.model.nv)
    controlWeights = np.array([0.1]*4 + [100]*6)
    
    goalTrackingCost = CostModelFramePlacement(rmodel,
                                               frame=rmodel.getFrameId(frameName),
                                               ref=SE3ref,
                                               nu =actModel.nu)
    xRegCost = CostModelState(rmodel, 
                              state, 
                              ref=state.zero(), 
                              nu=actModel.nu,
                              activation=ActivationModelWeightedQuad(stateWeights))
    uRegCost = CostModelControl(rmodel, 
                                nu=robot.
                                model.nv-2,
                                activation = ActivationModelWeightedQuad(controlWeights))
    uLimCost = CostModelControl(rmodel, 
                                nu=robot.
                                model.nv-2,
                                activation = ActivationModelInequality(np.array([0.1, 0.1, 0.1, 0.1, -1, -1, -1, -1, -1, -1]), 
                                                                    np.array([5, 5, 5, 5, 1, 1, 1, 1, 1, 1])))

    # Then let's add the running and terminal cost functions
    runningCostModel.addCost(name="pos", weight=0.1, cost=goalTrackingCost)
    runningCostModel.addCost(name="regx", weight=1e-4, cost=xRegCost)
    runningCostModel.addCost(name="regu", weight=1e-6, cost=uRegCost)
    runningCostModel.addCost(name="limu", weight=1e-3, cost=uLimCost)
    terminalCostModel.addCost(name="pos", weight=10, cost=goalTrackingCost)

    # DIFFERENTIAL ACTION MODEL
    dmodel = DifferentialActionModelActuated(rmodel, actModel, runningCostModel)
    model = IntegratedActionModelEuler(dmodel)
    model.timeStep =  integrationStep  
    return model   

In [13]:
# DEFINING THE SHOOTING PROBLEM & SOLVING

# Defining the time duration for running action models and the terminal one
dt = 5e-2

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 25


# DEFINE POSITION WAYPOINTS
target_pos = [np.array([0,0,1])]
target_pos += [np.array([0,1,1])]
quat = pin.Quaternion(1, 0, 0, 0)
quat.normalize()
target_quat = [quat]*2

# Plot goal frame
for i in range(0,len(target_pos)):
    robot.viewer.gui.addXYZaxis('world/wp%i' % i, [1., 0., 0., 1.], .03, 0.5)
    robot.viewer.gui.applyConfiguration('world/wp%i' % i, 
                                        target_pos[i].tolist() + [target_quat[i][0], target_quat[i][1], target_quat[i][2], target_quat[i][3]])
    
robot.viewer.gui.refresh()

In [16]:
models = []
for i in range(0,len(target_pos)):
#    if (i==1):
#        T *=2
    models += [uavPlacementModel(target_pos[i], target_quat[i], dt, 'base_link')]*T

q0 = robot.q0
x0 = np.hstack([m2a(q0), np.zeros(robot.model.nv)])

problem = ShootingProblem(x0, models[:-1], models[-1])

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverFDDP(problem)
ddp.callback = [CallbackDDPVerbose()]
ddp.callback.append(CallbackDDPLogger())

# Solving it with the DDP algorithm
ddp.solve()

ValueError: operands could not be broadcast together with shapes (10,) (4,) 

In [ ]:
displayTrajectory(robot, ddp.xs, dt)


In [17]:
robot.nv

6

In [ ]:
np.size(ddp.xs,0)

In [ ]:
distanceRotorCOG = 0.1525
cf = 6.6e-5
cm = 1e-6
pltUAM = PlotUAM(ddp.xs, ddp.us, np.size(ddp.us,0), dt, distanceRotorCOG, cf, cm)

fig, axs = pltUAM.plotMotorForces()
fig,axs = pltUAM.plotActuation()

In [ ]:
t = np.arange(0, 2*T*dt-dt, dt)
control = np.vstack(ddp.us)
fig = plt.figure()
axs = plt.plot(t, control[:,1])
axs.set_title('Moments')